# Analysis of Orbits in Fourier Space

Here we simulate Dan's Machine Learning system with rebound and then do a basic Fourier analysis. 

In [1]:
import rebound
import numpy as np
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

Here we set up the outer solar system with the possibility to inflate the masses. Such a system is known to go unstable on short timescales (e.g. Chambers, 1999). The simulation stops once there is a close encounter. 

In [ ]:
def problem_com(inner_massfac,output_N,tmax,seed):
    sim = rebound.Simulation()
    sim.integrator = "whfast"
    sim.dt = 1e-4*2*np.pi
    
    #parameters
    m_planet = 5*3e-6    #5-earth masses
    a_planet = 1
    rhill = a_planet*(m_planet/3.)**(1./3.)
    e = 1e-6
    
    np.random.seed(seed)
    
    #add bodies - COM frame
    sim.add(m=1.)
    sim.add(a=a_planet - 52.63*rhill,m=m_planet*inner_massfac,e=e,inc=1e-4,f=np.random.random()*np.pi*2)
    sim.add(a=a_planet,m=m_planet,e=e,inc=2e-4,f=np.random.random()*np.pi*2)
    sim.add(a=a_planet + 8*rhill,m=m_planet,e=e,inc=1e-5,f=np.random.random()*np.pi*2)
    sim.N_active = sim.N
    
    sim.move_to_com()
    E0 = sim.calculate_energy()
    #sim.init_megno()
    
    #condition to stop the simulation
    m_neptune = 5e-5
    sim.exit_min_distance = a_planet*(m_neptune/3.)**(1./3.)
    
    #integrate and output
    output_name = "Fourier/data/DTsys_t"+str('{0:.1e}'.format(tmax))+"_innerMF"+str(inner_massfac)+"_sd"+str(seed)+".csv"
    output_info = "Fourier/data/DTsys_t"+str('{0:.1e}'.format(tmax))+"_innerMF"+str(inner_massfac)+"_sd"+str(seed)+"_info.txt"
    start_time = time.time()
    times = np.linspace(0,tmax*2*np.pi,output_N)
    print 'starting simulation for '+output_name
    with open(output_name, 'w') as f:
        try:
            for t in times:
                sim.integrate(t)
                f.write(str(sim.t/(2*np.pi)))
                #f.write(','+str(sim.calculate_lyapunov()))
                outs = []
                for p in sim.particles[1:sim.N - sim.N_var]:
                    outs += [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]
                for out in outs:
                    f.write(',{0}'.format(out))
                f.write('\n')
            f.close()
            o = open(output_info,"w")
            o.write('Simulation complete. Final energy error is: '+str(abs((E0 - sim.calculate_energy())/E0))+".")
            o.write('\nTotal Elapsed time is: '+str(time.time() - start_time)+' seconds.')
            o.close()
            print 'Simulation '+output_name+' complete. Final energy error is:', abs((E0 - sim.calculate_energy())/E0)
        except rebound.Encounter as error:
            f.write(str(sim.t/(2*np.pi)))
            #f.write(','+str(sim.calculate_lyapunov()))
            outs = []
            for p in sim.particles[1:sim.N - sim.N_var]:
                outs += [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]
            outs += []
            for out in outs:
                f.write(',{0}'.format(out))
            f.write('\n')
            f.close()
            o = open(output_info,"w")
            o.write("Close encounter, simulation "+output_name+" stopping.")
            o.write('\nTotal Elapsed time is: '+str(time.time() - start_time)+' seconds.')
            o.close()
            print "Close encounter, simulation "+output_name+" stopping."

In [ ]:
def problem_helio(inner_massfac,output_N,tmax,seed):
    sim = rebound.Simulation()
    sim.integrator = "ias15"
    sim.dt = 1e-3*2*np.pi
    
    #parameters
    m_planet = 5*3e-6    #5-earth masses
    a_planet = 1
    rhill = a_planet*(m_planet/3.)**(1./3.)
    e = 0
    inc = 0
    
    np.random.seed(seed)
    
    #add bodies - helio frame
    sim.add(m=1.)
    sim.add(a=a_planet-52.63*rhill,m=m_planet*inner_massfac,e=e,inc=inc,f=np.random.random()*np.pi*2,primary=sim.particles[0])
    sim.add(a=a_planet,m=m_planet,e=e,inc=inc,f=np.random.random()*np.pi*2,primary=sim.particles[0])
    sim.add(a=a_planet+8*rhill,m=m_planet,e=e,inc=inc,f=np.random.random()*np.pi*2,primary=sim.particles[0])
    sim.N_active = sim.N
    
    sim.move_to_com()
    E0 = sim.calculate_energy()
    #sim.init_megno()
    
    #condition to stop the simulation
    m_neptune = 5e-5
    sim.exit_min_distance = a_planet*(m_neptune/3.)**(1./3.)
    
    #integrate and output
    output_name = "Fourier/data/DThelio_t"+str('{0:.1e}'.format(tmax))+"_innerMF"+str(inner_massfac)+"_sd"+str(seed)+".csv"
    output_info = "Fourier/data/DThelio_t"+str('{0:.1e}'.format(tmax))+"_innerMF"+str(inner_massfac)+"_sd"+str(seed)+"_info.txt"
    start_time = time.time()
    times = np.linspace(0,tmax*2*np.pi,output_N)
    print 'starting simulation for '+output_name
    with open(output_name, 'w') as f:
        try:
            for t in times:
                sim.integrate(t)
                f.write(str(sim.t/(2*np.pi)))
                #f.write(','+str(sim.calculate_lyapunov()))
                outs = []
                orbits = sim.calculate_orbits(heliocentric=True)
                for o in orbits:
                    outs += [o.a, o.P, o.e, o.pomega, o.inc, o.Omega, o.f]
                for out in outs:
                    f.write(',{0}'.format(out))
                f.write('\n')
            f.close()
            out = open(output_info,"w")
            out.write('Simulation complete. Final energy error is: '+str(abs((E0 - sim.calculate_energy())/E0))+".")
            out.write('\nTotal Elapsed time is: '+str(time.time() - start_time)+' seconds.')
            out.close()
            print 'Simulation '+output_name+' complete. Final energy error is:', abs((E0 - sim.calculate_energy())/E0)
        except rebound.Encounter as error:
            f.write(str(sim.t/(2*np.pi)))
            #f.write(','+str(sim.calculate_lyapunov()))
            outs = []
            orbits = sim.calculate_orbits(heliocentric=True)
            for o in orbits:
                outs += [o.a, o.P, o.e, o.pomega, o.inc, o.Omega, o.f]
            outs += []
            for out in outs:
                f.write(',{0}'.format(out))
            f.write('\n')
            f.close()
            out = open(output_info,"w")
            out.write("Close encounter, simulation "+output_name+" stopping.")
            out.write('\nTotal Elapsed time is: '+str(time.time() - start_time)+' seconds.')
            out.close()
            print "Close encounter, simulation "+output_name+" stopping."

Choose the parameters you will search over.

In [ ]:
n_runs = 10             #number of simulations you want to run
max_inner_massfac = 1   #max mass factor to increase inner planet by (60 = Jupiter)
n_outputs = 5e4         #number of outputted data points, important for FFT
tmax = 1e6              #simulation time

**Warning!** Executing the code below will take a long time!

In [ ]:
#Get inputs
massfac = np.round(np.logspace(0,np.log10(max_inner_massfac),n_runs),2)
params = zip(massfac,n_outputs*np.ones(n_runs),tmax*np.ones(n_runs),(1000*np.random.random(n_runs)).astype("int"))

#Run the problem
import multiprocessing as mp
def execute(pars):
    problem_helio(pars[0],pars[1],pars[2],pars[3])

if __name__== '__main__':
    pool = mp.Pool(processes=n_runs)
    pool.map(execute, params)
    pool.close()
    pool.join()

Now go to Fourier_Analyze notebook to analyze the results.